In [2]:
import os
import pandas as pd
from pydantic import Field
from pydantic_settings import BaseSettings, SettingsConfigDict
# from semantic_router.encoders import OpenAIEncoder
# from encoder import OpenAIEncoder

In [4]:
class Settings(BaseSettings):
    model_config = SettingsConfigDict(
        env_file="../.env", env_file_encoding="utf-8", extra="ignore"
    )
    embedding_base_url: str
    embedding_api_key: str
    embedding_model: str

settings = Settings()
print(settings.embedding_model)

baai/bge-m3


# Prepare Embedder & DB

In [5]:
import os
from langchain_openai import OpenAIEmbeddings

os.environ["OPENAI_API_BASE"] = settings.embedding_base_url

In [7]:
embeddings = OpenAIEmbeddings(
    model=settings.embedding_model,
    api_key=settings.embedding_api_key
)

In [10]:
vectors = embeddings.embed_documents(["hello", "goodbye"])
len(vectors[0])

1024

In [11]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

In [13]:
client = QdrantClient(":memory:")

client.create_collection(
    collection_name="demo_collection",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)
vector_store = QdrantVectorStore(
    client=client,
    collection_name="demo_collection",
    embedding=embeddings,
)

In [14]:
vector_store

# Test Document

In [19]:
from uuid import uuid4
from langchain_core.documents import Document

In [20]:
document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)
documents = [
    document_1,
    document_2
]
uuids = [str(uuid4()) for _ in range(len(documents))]

In [21]:
vector_store.add_documents(documents=documents, ids=uuids)

['c96f8340-b911-4849-8628-5e3a62323e33',
 '5ba950c8-7713-45e1-ac94-515bd635dbfb']

In [ ]:
## delete
# vector_store.delete(ids=[uuids[-1]])

# Test Search

In [22]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy", k=2
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* I had chocalate chip pancakes and scrambled eggs for breakfast this morning. [{'source': 'tweet', '_id': 'c96f8340-b911-4849-8628-5e3a62323e33', '_collection_name': 'demo_collection'}]
* The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news', '_id': '5ba950c8-7713-45e1-ac94-515bd635dbfb', '_collection_name': 'demo_collection'}]


## vector search

In [23]:
from langchain_qdrant import RetrievalMode

qdrant = QdrantVectorStore.from_documents(
    documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="my_documents",
    retrieval_mode=RetrievalMode.DENSE,
)

In [25]:
query = "What did the president say about Ketanji Brown Jackson"
found_docs = qdrant.similarity_search(query)
found_docs

[Document(metadata={'source': 'news', '_id': '14cd9d0d9bf647fcb80819c5be43b732', '_collection_name': 'my_documents'}, page_content='The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.'),
 Document(metadata={'source': 'tweet', '_id': 'f450988d7afc437f92e5206e4b9083d0', '_collection_name': 'my_documents'}, page_content='I had chocalate chip pancakes and scrambled eggs for breakfast this morning.')]

## Hybrid

In [28]:
from langchain_qdrant import FastEmbedSparse, RetrievalMode
sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")

qdrant = QdrantVectorStore.from_documents(
    documents,
    embedding=embeddings,
    sparse_embedding=sparse_embeddings,
    location=":memory:",
    collection_name="my_documents",
    retrieval_mode=RetrievalMode.HYBRID,
)

Fetching 29 files:   0%|          | 0/29 [00:00<?, ?it/s]

arabic.txt:   0%|          | 0.00/6.35k [00:00<?, ?B/s]

danish.txt:   0%|          | 0.00/424 [00:00<?, ?B/s]

catalan.txt:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

dutch.txt:   0%|          | 0.00/453 [00:00<?, ?B/s]

...

slovene.txt:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

swedish.txt:   0%|          | 0.00/559 [00:00<?, ?B/s]

spanish.txt:   0%|          | 0.00/2.18k [00:00<?, ?B/s]

tajik.txt:   0%|          | 0.00/1.82k [00:00<?, ?B/s]

turkish.txt:   0%|          | 0.00/260 [00:00<?, ?B/s]

In [30]:
query = "What did the president say about Ketanji Brown Jackson"
found_docs = qdrant.similarity_search(query)
found_docs

[Document(metadata={'source': 'news', '_id': 'ada96de7c1ad4e0cb94576cb93e4abff', '_collection_name': 'my_documents'}, page_content='The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.'),
 Document(metadata={'source': 'tweet', '_id': 'bbd47c8db070443cb7c882baffcff815', '_collection_name': 'my_documents'}, page_content='I had chocalate chip pancakes and scrambled eggs for breakfast this morning.')]

## Metadata Filtering

In [31]:
from qdrant_client import models

results = vector_store.similarity_search(
    query="Who are the best soccer players in the world?",
    k=1,
    filter=models.Filter(
        should=[
            models.FieldCondition(
                key="page_content",
                match=models.MatchValue(
                    value="The top 10 soccer players in the world right now."
                ),
            ),
        ]
    ),
)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")